# Chapter 15

In [ ]:
%use s2

import dev.nm.algebra.linear.matrix.doubles.operation.MatrixFactory.cbind
import dev.nm.stat.cointegration.JohansenAsymptoticDistribution.Test
import java.util.Arrays

In [ ]:
println("Chapter 15 demos")

println("Convert VAR to VECM")

// define a VAR(2) model
val varxModel = VARXModel(
        arrayOf(
            DenseMatrix(
                    arrayOf(
                        doubleArrayOf(-0.210, 0.167),
                        doubleArrayOf(0.512, 0.220)
                    )),
            DenseMatrix(
                    arrayOf(
                        doubleArrayOf(0.743, 0.746),
                        doubleArrayOf(-0.405, 0.572)
                    ))
        ),
        null)

// construct a VECM fron a VAR
val vecm = VECMTransitory(varxModel)

println("dimension = " + vecm.dimension())
println("PI, the impact matrix = ")
println(vecm.pi())
println("GAMMA = ")
println(vecm.gamma(1))

In [ ]:
println("Cointegration between SPX and AAPL")

// read the monthly S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_monthly.csv", true, true)

// convert the csv file into a matrix for manipulation
val spx_M1: Matrix = DenseMatrix(spx_arr)
// remove the data before 2008
val spx_M2: Matrix = MatrixFactory.subMatrix(spx_M1, 325, spx_M1.nRows(), 1, spx_M1.nCols())
// extract the column of adjusted close prices
val spx_v: Vector = spx_M2.getColumn(5) // adjusted closes

// read the monthly AAPL data from a csv file
val appl_arr = DoubleUtils.readCSV2d("./resources/AAPL_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val aapl_M1: Matrix = DenseMatrix(appl_arr)
// remove the data before 2008
val aapl_M2: Matrix = MatrixFactory.subMatrix(aapl_M1, 325, aapl_M1.nRows(), 1, aapl_M1.nCols())
// extract the column of adjusted close prices
val aapl_v: Vector = aapl_M2.getColumn(5) // adjusted closes

// combine SPX and AAPL to form a bivariate time series
val mts = MultivariateSimpleTimeSeries(cbind(spx_v, aapl_v))
//      println("(spx, aapl) prices: \n" + mts)

// run cointegration on all combinations of available test and trend types
for (test: Test in Test.values()) {
    for (trend: dev.nm.stat.cointegration.JohansenAsymptoticDistribution.TrendType in dev.nm.stat.cointegration.JohansenAsymptoticDistribution.TrendType.values()) {
        val coint = CointegrationMLE(mts, true)
        val johansen = JohansenTest(test, trend, coint.getEigenvalues().size())
        println("alpha:")
        println(coint.alpha())
        println("beta:")
        println(coint.beta())
        println("Johansen test: "
                + test.toString()
                + "\t" + trend.toString()
                + "\t eigenvalues: " + coint.getEigenvalues()
                + "\t statistics: " + johansen.getStats(coint)
        )
    }
}

// run ADF test to check if the cointegrated series is indeed stationary
val betas: DoubleArray = doubleArrayOf(-12.673296, -6.995392)
for (beta: Double in betas) {
    println(String.format("testing for beta = %f%n", beta))
    val ci: Vector = spx_v.add(aapl_v.scaled(beta))
    val adf = AugmentedDickeyFuller(
                    ci.toArray(),
                    TrendType.CONSTANT, // constant drift term
                    4, // the lag order
                    null
            )

    println("H0: " + adf.getNullHypothesis())
    println("H1: " + adf.getAlternativeHypothesis())
    println(String.format("the p-value for the test = %f%n", adf.pValue()))
    println(String.format("the statistics for the test = %f%n", adf.statistics()))
}

In [ ]:
println("Varim simulation")

// number of random numbers to generate
val T = 100000

// the mean
val MU: Vector = DenseVector(1.0, 1.0)
// the AR coefficients
val PHI = arrayOf(
    DenseMatrix(
    arrayOf(
        doubleArrayOf(0.5, 0.5),
        doubleArrayOf(0.0, 0.5)
)))
// the MA coefficients
val THETA = arrayOf(PHI[0])
// the white noise covariance structure
val SIGMA: Matrix = DenseMatrix(
                arrayOf(
                    doubleArrayOf(1.0, 0.2),
                    doubleArrayOf(0.2, 1.0)
                ))

// construct a VARMA model
val VARMA = VARMAModel(MU, PHI, THETA, SIGMA)

// construct a random number generator from a VARMA model
val SIM = VARIMASim(VARMA)
SIM.seed(1234567890L)

// produce the random vectors
val data = arrayOfNulls<DoubleArray>(T)
for (i in 0 until T) {
    data[i] = SIM.nextVector()
}

// statistics about the simulation
// println(String.format("each vector size = %d%n", data[0].size()))
// println(String.format("sample size = %d%n", data.length))

// compute the theoretical mean of the data
// val theoretical_mean: DoubleArray = Inverse(DenseMatrix(arrayOf(doubleArrayOf(2.0, 2.0))).ONE().minus(VARMA.AR(1))).multiply(DenseVector(MU)).toArray()
// println("theoretical mean =")
// println(Arrays.toString(theoretical_mean))

// cast the random data in matrix form for easy manipulation
val dataM: Matrix = MatrixFactory.rbind(DenseMatrix(data))
// compute the sample mean of the data
val sample_mean1: Double = Mean(dataM.getColumn(1).toArray()).value()
val sample_mean2: Double = Mean(dataM.getColumn(2).toArray()).value()
println(String.format("sample mean of the first variable = %f%n", sample_mean1))
println(String.format("sample mean of the second variable = %f%n", sample_mean2))

// compute the theoretical covariance of the data
val cov_theoretical: Matrix = DenseMatrix(
        arrayOf(
            doubleArrayOf(811.0 / 135.0, 101.0 / 45.0),
            doubleArrayOf(101.0 / 45.0, 7.0 / 3.0)
        ))
println("theoretical covariance =")
println(cov_theoretical)

// compute the sample covariance of the data
val sample_cov = SampleCovariance(dataM)
println("sample covariance =")
println(sample_cov)

In [ ]:
println("Autocovariance function for a causal VARMA")

// the AR coefficients
val AR = arrayOf(
    DenseMatrix(
        arrayOf(
            doubleArrayOf(0.5, 0.5),
            doubleArrayOf(0.0, 0.5)))
)

// the MA coefficients
var MA = arrayOf(
    AR[0].t()
)

// SIGMA
val SIGMA: Matrix = DenseMatrix(
                arrayOf(
                    doubleArrayOf(1.0, 0.2),
                    doubleArrayOf(0.2, 1.0)
                ))

// define a VARIMA(1,1) model
val varma11 = VARMAModel(AR, MA, SIGMA)

// comptue the autocovariance function for a VARIMA process up to a certian number of lags
val GAMMA = VARMAAutoCovariance(
                varma11,
                10 // number of lags
        )

// print out the autocovariance function 
for (i in 0..5) {
    println(String.format("GAMMA(%d) = %n", i))
    println(GAMMA.evaluate(1.0))
    println()
}

In [ ]:
/**
* P. J. Brockwell and R. A. Davis, "Example 5.2.1. Chapter 5. Multivariate
* Time Series," in <i>Time Series: Theory and Methods</i>, Springer, 2006.
*/

println("Forecast using the innovation algorithm")

// MA(1) model parameters
val theta: Double = -0.9
val sigma: Double = 1.0

// a multivariate time series (although the dimension is just 1)
val Xt = MultivariateSimpleTimeSeries(
            DenseMatrix(
                arrayOf(
                    doubleArrayOf(-2.58),
                    doubleArrayOf(1.62),
                    doubleArrayOf(-0.96),
                    doubleArrayOf(2.62),
                    doubleArrayOf(-1.36)
                ))
            )

// the autocovariance function
val K = object : MultivariateAutoCovarianceFunction() {

    override fun evaluate(x1: Double, x2: Double): Matrix {
        val i: Int = x1.toInt()
        val j: Int = x2.toInt()

        var k: Double = 0.0

        if (i == j) {
            k = sigma * sigma
            k *= 1 + theta * theta
        }

        if (Math.abs(j - i) == 1) {
            k = theta
            k *= sigma * sigma
        }

        // γ = 0 otherwise
        val result: DenseMatrix = DenseMatrix(arrayOf(doubleArrayOf(k)))
        return result
    }
}

// run the innovation algorithm
val forecast = MultivariateForecastOneStep(Xt, K)

// making forecasts
for (i in 0..5) {
    println(Arrays.toString(forecast.xHat(i).toArray()))
}

In [ ]:
println("VAR models")

// construct a VAR(2) model
val MU: Vector = DenseVector(1.0, 2.0)
val PHI = arrayOf(
    DenseMatrix(
        arrayOf(
            doubleArrayOf(0.2, 0.3),
            doubleArrayOf(0.0, 0.4))),
    DenseMatrix(
        arrayOf(
            doubleArrayOf(0.1, 0.2),
            doubleArrayOf(0.3, 0.1)))
)
val model0 = VARModel(MU, PHI)

// construct a RNG from the model
val sim = VARIMASim(model0)
sim.seed(1234567891L)

// generate a random multivariate time series
val N = 5000
var ts = arrayOfNulls<DoubleArray>(N)
for (i in 0 until N) {
    ts[i] = sim.nextVector()
}
val mts = MultivariateSimpleTimeSeries(DenseMatrix(ts))

// fit the data to a VAR(2) model
val model1 = VARFit(mts, 2)
println("μ = ")
println(model1.mu())
println("ϕ_1 =")
println(model1.AR(1))
println("ϕ_2 =")
println(model1.AR(2))
println("sigma =")
println(model1.sigma())

In [ ]:
println("VAR, VMA, VARMA, VARIMA and all those")

// construct a VAR(1) model
val PHI: Array<Matrix?> = arrayOfNulls<Matrix>(1)
PHI[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.7, 0.12),
            doubleArrayOf(0.31, 0.6)
        ))
val var1 = VARModel(PHI)
println("unconditional mean = " + var1.unconditionalMean())

// construct a VAR(1) model
val THETA: Array<Matrix?> = arrayOfNulls<Matrix>(1)
THETA[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.5, 0.16),
            doubleArrayOf(-0.7, 0.28)
        ))
val vma1 = VMAModel(THETA)
println("unconditional mean = " + vma1.unconditionalMean())

// construct a VARIMA(1,1) model
val PHI1: Array<Matrix?> = arrayOfNulls<Matrix>(1)
PHI1[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.7, 0.0),
            doubleArrayOf(0.0, 0.6)
        ))
val THETA1: Array<Matrix?> = arrayOfNulls<Matrix>(1)
THETA1[0] = DenseMatrix(
        arrayOf(
            doubleArrayOf(0.5, 0.6),
            doubleArrayOf(-0.7, 0.8)
        ))

val varma11: VARMAModel = VARMAModel(
        PHI1,
        THETA1
)
println("unconditional mean = " + varma11.unconditionalMean())

// the AR coefficients
// val PHI: Array<Matrix?> = arrayOfNulls<Matrix>(4)
// PHI[0] = DenseMatrix(DenseVector(arrayOf(0.3)))
// PHI[1] = DenseMatrix(DenseVector(arrayOf(-0.2)))
// PHI[2] = DenseMatrix(DenseVector(arrayOf(0.05)))
// PHI[3] = DenseMatrix(DenseVector(arrayOf(0.04)))

// the MA coefficients
// val THETA: Array<Matrix?> = arrayOfNulls<Matrix>(2)
// THETA[0] = DenseMatrix(DenseVector(arrayOf(0.2)))
// THETA[1] = DenseMatrix(DenseVector(arrayOf(0.5)))

// the order of integration
// val d = 1

// construct a VARIMA(1,1,1) model
// val varima111 = VARIMAModel(PHI_1, d, THETA_1)

In [ ]:
println("SPX and APPL")

// read the monthly S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val spx_M: Matrix = DenseMatrix(spx_arr)
// extract the column of adjusted close prices
val spx_v: Vector = spx_M.getColumn(5) // adjusted closes

// read the monthly AAPL data from a csv file
val appl_arr = DoubleUtils.readCSV2d("./resources/AAPL_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val aapl_M: Matrix = DenseMatrix(appl_arr)
// extract the column of adjusted close prices
val aapl_v: Vector = aapl_M.getColumn(5) // adjusted closes

// combine SPX and AAPL to form a bivariate time series
val mts = MultivariateSimpleTimeSeries(cbind(spx_v, aapl_v))
println("(spx, aapl) prices: \n" + mts)

// compute the bivariate time series of log returns
val p1: Matrix = mts.drop(1).toMatrix()
val p: Matrix = mts.toMatrix()
var log_returns_M: Matrix = p1.ZERO() // allocate space for the matrix
for (i in 1 until p1.nRows()) { // matrix and vector index starts from 1
    var r: Double = log10(p1.get(i, 1)) - log10(p.get(i, 1))
    log_returns_M.set(i, 1, r)
    r = log10(p1.get(i, 2)) - log10(p.get(i, 2))
    log_returns_M.set(i, 2, r)
}
// convert a matrix to a multivariate time series
val log_returns = MultivariateSimpleTimeSeries(log_returns_M)
println("(spx, aapl) log_returns: \n" + log_returns)

// fit the log returns to an VAR(1) model
val fit = VARFit(log_returns, 1)
println("the estimated phi_1 for var(1) is")
println(fit.AR(1))
val log_returns2 = fit.getDemeanedModel() // demeaned version

// predict the future values using the innovation algorithm
val K = VARMAAutoCovariance(log_returns2, log_returns.size())
val forecast = MultivariateForecastOneStep(log_returns, K)
for (i in 0..5) {
    println(Arrays.toString(forecast.xHat(i).toArray()))
}

In [ ]:
println("Multivariate time series")

// construct a bivariate time series
val X_T0 = MultivariateSimpleTimeSeries(
                DenseMatrix(
                    arrayOf(
                        doubleArrayOf(-1.875, 1.693),
                        doubleArrayOf(-2.518, -0.03),
                        doubleArrayOf(-3.002, -1.057),
                        doubleArrayOf(-2.454, -1.038),
                        doubleArrayOf(-1.119, -1.086),
                        doubleArrayOf(-0.72, -0.455),
                        doubleArrayOf(-2.738, 0.962),
                        doubleArrayOf(-2.565, 1.992),
                        doubleArrayOf(-4.603, 2.434),
                        doubleArrayOf(-2.689, 2.118)
                )))
println("X_T0: " + X_T0)

// first difference of the bivariate time series
val X_T1 = X_T0.diff(1)
println("diff(1): " + X_T1)

// first order lagged time series
val X_T2 = X_T0.lag(1)
println("lag(1): " + X_T2)

// drop the first two items
val X_T3 = X_T0.drop(1)
println("drop(1): " + X_T3)

In [ ]:
println("ARCH and GARCH models")

// σ_t^2 = 0.2 + 0.212 * ε_(t-1)^2
val arch1 = GARCHModel(
                0.2, // constant
                doubleArrayOf(0.212), // ARCH terms
                doubleArrayOf() // no GARCH terms
        )
println(arch1)
println(String.format("conditional variance = %f%n", arch1.`var`()))

// σ_t^2= 0.2+0.212ε_(t-1)^2+0.106σ_(t-1)^2
val garch11 = GARCHModel(
                0.2, // constant
                doubleArrayOf(0.212), // ARCH terms
                doubleArrayOf(0.106) // GARCH terms
        )
println(garch11)
println(String.format("conditional variance = %f%n", garch11.`var`()))

// simulate the GARCH process
val sim = GARCHSim(garch11)
sim.seed(1234567890L)
val series: DoubleArray = RNGUtils.nextN(sim, 200)
println(Arrays.toString(series))

In [ ]:
/**
*
* x<-c(0.2,0.3,-0.1,0.4,-0.5,0.6,0.1,0.2) adf.test(c(x,x),alternative =
* c("stationary"),4)
*
* The p-value from R is obtained form the interpolation
* (0.9-0.1)/(3.24-1.14)=(x-0.1)/(3.24-2.642)
* "x=0.8/(3.24-1.14)*(3.24-2.642)+0.1=0.3278"
*
* The values -3.24 and -1.14 are from table 4.2c in A. Banerjee, J. J.
* Dolado, J. W. Galbraith, and D. F. Hendry (1993): Cointegration, Error
* Correction, and the Econometric Analysis of Non-Stationary Data, Oxford
* University Press, Oxford.
*/

println("unit root test using the augmented Dickey-Fuller test")

val sample: DoubleArray = doubleArrayOf(0.2, 0.3, -0.1, 0.4, -0.5, 0.6, 0.1, 0.2, 0.2, 0.3, -0.1, 0.4, -0.5, 0.6, 0.1, 0.2)
val adf = AugmentedDickeyFuller(
                sample,
                TrendType.CONSTANT, // constant drift term
                4, // the lag order
                null
        )

println("H0: " + adf.getNullHypothesis())
println("H1: " + adf.getAlternativeHypothesis())
println(String.format("the p-value for the test = %f%n", adf.pValue()))
println(String.format("the statistics for the test = %f%n", adf.statistics()))

In [ ]:
println("Check the goodness of fit using the Ljung-Box test")

// define an ARMA(1, 1)
val arma11 = ARMAModel(
        doubleArrayOf(0.2), // the AR coefficients
        doubleArrayOf(1.1) // the MA coefficients
)

// create a random number generator from the ARMA model
val sim = ARIMASim(arma11)
sim.seed(1234567890L)

// generate a random time series
val T = 50 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}

// fit an ARMA(1,1) model
val css = ConditionalSumOfSquares(x, 1, 0, 1)
val arma11_css = css.getARMAModel()
println(String.format("The fitted ARMA(1,1) model is %s%n", arma11_css))
println(String.format("The fitted ARMA(1,1) model, demeanded, is %s%n", arma11_css.getDemeanedModel()))
println(String.format("AIC = %f%n", css.AIC()))

// compute the fitted values using the fitted model
val forecaset = ARMAForecastOneStep(x, arma11_css)
var x_hat: DoubleArray = DoubleArray(T)
var residuals: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // the fitted value
    x_hat[i] = forecaset.xHat(i)
    // the residuals
    residuals[i] = x[i] - x_hat[i]
}

println(Arrays.toString(x))
println(Arrays.toString(x_hat))
println(Arrays.toString(residuals))

// run the goodness-of-fit test
val test = LjungBox(
        residuals,
        4, // check up to the 4-th lag
        2 // number of parameters
)
println(String.format("The test statistic = %f%n", test.statistics()))
println(String.format("The p-value = %f%n", test.pValue()))

In [ ]:
println("Making forecasts for an ARMA model")

// define an ARMA(1, 1)
val arma11 = ARMAModel(
        doubleArrayOf(0.2), // the AR coefficients
        doubleArrayOf(1.1) // the MA coefficients
)

// create a random number generator from the ARMA model
val sim = ARIMASim(arma11)
sim.seed(1234567890L)

// generate a random time series
val T = 50 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}

// compute the one-step conditional expectations for the model
val forecaset = ARMAForecastOneStep(x, arma11)
var x_hat: DoubleArray = DoubleArray(T)
var residuals: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // the one-step conditional expectations
    x_hat[i] = forecaset.xHat(i)
    // the errors
    residuals[i] = x[i] - x_hat[i]
}

println(Arrays.toString(x))
println(Arrays.toString(x_hat))
println(Arrays.toString(residuals))

In [ ]:
println("ARMA(2,3) model")

// build an ARMA(2, 3)
val arma23 = ARMAModel(
        doubleArrayOf(0.6, -0.23), // the AR coefficients
        doubleArrayOf(0.1, 0.2, 0.4) // the MA coefficients
)

// compute the linear representation
val ma = LinearRepresentation(arma23)
for (i in 1..20) {
    println(String.format("the coefficients of the linear representation at lag %d = %f%n", i, ma.AR(i)))
}

// compute the autocovariance function for the model
val acvf = AutoCovariance(arma23)
for (i in 1 until 10) {
    println(String.format("the acvf of the ARMA(2,3) model at lag%d: %f%n", i, acvf.evaluate(i.toDouble())))
}

// compute the autocorrelation function for the model
val acf = AutoCorrelation(arma23, 10)
for (i in 1 until 10) {
    println(String.format("the acf of the ARMA(2,3) model at lag%d: %f%n", i, acf.evaluate(i.toDouble())))
}

In [ ]:
println("ARMA(1,1) model")

// build an ARMA(1, 1)
val arma11 = ARMAModel(
        doubleArrayOf(0.2), // the AR coefficients
        doubleArrayOf(1.1) // the MA coefficients
)

// compute the linear representation
val ma = LinearRepresentation(arma11)
for (i in 1..20) {
    println(String.format("the coefficients of the linear representation at lag %d = %f%n", i, ma.AR(i)))
}

// compute the autocovariance function for the model
val acvf = AutoCovariance(arma11)
for (i in 1 until 10) {
    println(String.format("the acvf of the ARMA(1,1) model at lag%d: %f%n", i, acvf.evaluate(i.toDouble())))
}

// compute the autocorrelation function for the model
val acf = AutoCorrelation(arma11, 10)
for (i in 1 until 10) {
    println(String.format("the acf of the ARMA(1,1) model at lag%d: %f%n", i, acf.evaluate(i.toDouble())))
}


In [ ]:
println("MA models")

// define an MA(1) model
val ma1 = MAModel(
        doubleArrayOf(0.8) // theta_1 = 0.8
)

val nLags = 10
// compute the autocovariance function for the model
val acvf1 = AutoCovariance(ma1)
for (i in 0 until nLags) {
    println(String.format("the acvf of the MA(1) model at lag%d = %f%n", i, acvf1.evaluate(i.toDouble())))
}
// compute the autocorrelation function for the model
val acf1 = AutoCorrelation(ma1, nLags)
for (i in 0 until nLags) {
    println(String.format("the acf of the MA(1) model at lag%d = %f%n", i, acf1.evaluate(i.toDouble())))
}

// define an MA(2) model
val ma2 = MAModel(
        doubleArrayOf(-0.2, 0.01) // the moving-average coefficients
)
val acvf2 = AutoCovariance(ma2)
for (i in 1 until 10) {
    println(String.format("the acvf of the MA(2) model at lag%d: %f%n", i, acvf2.evaluate(i.toDouble())))
}
val acf2 = AutoCorrelation(ma2, 10)
for (i in 0 until 10) {
    println(String.format("the acf of the MA(2) model at lag%d: %f%n", i, acf2.evaluate(i.toDouble())))
}

// define an MA(2) model
val ma3 = MAModel(
        0.1, // the mean
        doubleArrayOf(-0.5, 0.01), // the moving-average coefficients
        10.0 // the standard deviation
)

// create a random number generator from the MA model
val sim = ARIMASim(ma3)
sim.seed(1234567890L)

// generate a random time series
val T = 500 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}

// determine the cutoff lag using autocorrelation
val acf3_hat = SampleAutoCorrelation(SimpleTimeSeries(x))
for (i in 0 until 5) {
    println(String.format("the empirical acf of a time series at lag%d: %f%n", i, acf3_hat.evaluate(i)))
}

// fit an MA(2) model using the data
val fit = ConditionalSumOfSquares(
                x,
                0,
                0,
                2) // the MA order
println(String.format("theta: %s%n", Arrays.toString(fit.getARMAModel().theta())))
println(String.format("var of error term: %s%n", fit.`var`()))

// make forecast
val forecast_ma3 = ARMAForecast(SimpleTimeSeries(x), ma3)
println("The forecasts for the MA(2) model are")
for (j in 0 until 10) {
    println(forecast_ma3.next())
}

In [ ]:
println("AR models")

// define an AR(1) model
val ar1 = ARModel(
        doubleArrayOf(0.6) // phi_1 = 0.6
)
val nLags = 10
// compute the autocovariance function for the model
val acvf1 = AutoCovariance(ar1)
for (i in 0 until nLags) {
    println(String.format("the acvf of the AR(1) model at lag%d = %f%n", i, acvf1.evaluate(i.toDouble())))
}
// compute the autocorrelation function for the model
val acf1 = AutoCorrelation(ar1, nLags)
for (i in 0 until nLags) {
    println(String.format("the acf of the AR(1) model at lag%d = %f%n", i, acf1.evaluate(i.toDouble())))
}

// define an AR(2) model
val ar2 = ARModel(
        doubleArrayOf(1.2, -0.35)
)
// compute the autocovariance function for the model
val acvf2 = AutoCovariance(ar2)
for (i in 1 until nLags) {
    println(String.format("the acvf of the AR(2) model at lag%d = %f%n", i, acvf2.evaluate(i.toDouble())))
}
// compute the autocorrelation function for the model
val acf2 = AutoCorrelation(ar2, 10)
for (i in 0 until nLags) {
    println(String.format("the acf of the AR(2) model at lag%d = %f%n", i, acf2.evaluate(i.toDouble())))
}

// create a random number generator from the AR model
val sim = ARIMASim(ar2)
sim.seed(1234567890L)

// generate a random time series
val T = 500 // length of the time series
var x: DoubleArray = DoubleArray(T)
for (i in 0 until T) {
    // call the RNG to generate random numbers according to the specification
    x[i] = sim.nextDouble()
}

// determine the cutoff lag using partial autocorrelation
val acf2_hat = SamplePartialAutoCorrelation(SimpleTimeSeries(x))
for (i in 1 until 5) {
    println(String.format("the empirical pacf of a time series at lag%d: %f%n", i, acf2_hat.evaluate(i)))
}

// fit an AR(2) model using the data
val fit = ConditionalSumOfSquares(
                x,
                2, // the AR order
                0,
                0)
println(String.format("phi: %s%n", Arrays.toString(fit.getARMAModel().phi())))
println(String.format("var of error term: %s%n", fit.`var`()))

// make forecast
val forecast_ar2 = ARMAForecast(SimpleTimeSeries(x), ar2)
println("The forecasts for the AR(2) model are")
for (j in 0 until 10) {
    println(forecast_ar2.next())
}

In [ ]:
println("Classical time series decomposition")

val rng = StandardNormalRNG()

// construct an additive model
val additive_model = AdditiveModel(
        // the trend
        doubleArrayOf(
            1.0, 3.0, 5.0, 7.0, 9.0, 11.0, 13.0, 15.0, 17.0, 19.0
        ),
        // the seasonal component
        doubleArrayOf(
            0.0, 1.0
        ),
        // the source of randomness
        rng
)
println(additive_model)

// construct a multiplicative model
val multiplicative_model = MultiplicativeModel(
        // the trend
        doubleArrayOf(
            1.0, 3.0, 5.0, 7.0, 9.0, 11.0, 13.0, 15.0, 17.0, 19.0
        ),
        // the seasonal component
        doubleArrayOf(
            -1.0, 1.0
        ),
        // the source of randomness
        rng
)
println(multiplicative_model)

In [ ]:
println("Univariate time series of S&P 500 monthly close")

// read the monthly S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_monthly.csv", true, true)
// convert the csv file into a matrix for manipulation
val spx_M: Matrix = DenseMatrix(spx_arr)
// extract the column of adjusted close prices
val spx_v: Vector = spx_M.getColumn(5) // adjusted closes
// construct a univariate time series from the data
val spx = SimpleTimeSeries(spx_v.toArray())
//      println(spx)

// ACVF for lags 0, 1, 2, ..., 24
val acvf = SampleAutoCovariance(spx)
for (i in 0 until 25) {
    println(String.format("acvf(%d) = %f%n", i, acvf.evaluate(i)))
}

// ACF for lags 0, 1, 2, ..., 24
val acf = SampleAutoCorrelation(spx)
for (i in 0 until 25) {
    println(String.format("acf(%d) = %f%n", i, acf.evaluate(i)))
}

// PACF for lags 1, 2, ..., 24
val pacf = SamplePartialAutoCorrelation(spx)
for (i in 1 until 25) {
    println(String.format("pacf(%d) = %f%n", i, pacf.evaluate(i)))
}

// decompose the S&P 500 monthly returns into a classical additive model
val spx_model = MADecomposition(
                spx.toArray(), // the SP 500 monthly returns
                12 // the lenght of seasonality
        )
println(String.format("the trend component: %s%n", Arrays.toString(spx_model.getTrend())))
println(String.format("the seasonal component: %s%n", Arrays.toString(spx_model.getSeasonal())))
println(String.format("the random component: %s%n", Arrays.toString(spx_model.getRandom())))

// auto fit an ARIMA model to S&P 500 prices
val fit_sp500_1 = AutoARIMAFit(spx.toArray(), 5, 1, 5, 0, 0, 1000)
val arima1 = fit_sp500_1.optimalModelByAIC()
val arima2 = fit_sp500_1.optimalModelByAICC()
println("The optimal model for S&P500 prices by AIC is:")
println(arima1)
println("The optimal model for S&P500 prices by AICc is:")
println(arima2)

// compute log returns from prices
val p1: DoubleArray = spx.drop(1).toArray()
val p: DoubleArray = spx.toArray()
var log_returns: DoubleArray = DoubleArray(p1.size)
for (i in 0 until p1.size) {
    log_returns[i] = log10(p1[i]) - log10(p[i])
}
println(Arrays.toString(log_returns))

// auto fit an ARIMA model to S&P 500 log returns
val fit_sp500_2 = AutoARIMAFit(log_returns, 5, 1, 5, 0, 0, 1000)
val arima3 = fit_sp500_2.optimalModelByAIC()
val arima4 = fit_sp500_2.optimalModelByAICC()
println("The optimal model for S&P500 log returns by AIC is:")
println(arima3)
println("The optimal model for S&P500 log returns by AICc is:")
println(arima4)

// make price forecasts based on the fitted model
val forecast_prices = ARIMAForecast(spx, arima1)
println("The next 10 price forecasts for sp500 using the ARIMA(2,1,1) model is:")
for (i in 0 until 10) {
    println(forecast_prices.next())
}

// make log return forecasts based on the fitted model
val forecast_returns = ARIMAForecast(SimpleTimeSeries(log_returns), arima4)
println("The next 10 log return forecasts for sp500 using the ARMA(2, 3) model is:")
for (i in 0 until 10) {
    println(forecast_returns.next())
}

In [ ]:
println("Univariate time series of S&P 500 daily close")

// read the daily S&P 500 data from a csv file
val spx_arr = DoubleUtils.readCSV2d("./resources/sp500_daily.csv", true, true)
// convert the csv file into a matrix for manipulation
val spx_M: Matrix = DenseMatrix(spx_arr)
// extract the column of adjusted close prices
val spx_v: Vector = spx_M.getColumn(5) // adjusted closes
// construct a univariate time series from the data
val spx = SimpleTimeSeries(spx_v.toArray())
println(spx)

// compute simple returns from prices
val diff: DoubleArray = spx.diff(1).toArray()
val p0: DoubleArray = spx.lag(1).toArray()
var simple_returns: DoubleArray = DoubleArray(diff.size)
for (i in 0 until diff.size) {
    simple_returns[i] = diff[i] / p0[i]
}
println(Arrays.toString(simple_returns))

// compute log returns from prices
val p1: DoubleArray = spx.drop(1).toArray()
val p: DoubleArray = spx.toArray()
var log_returns: DoubleArray = DoubleArray(p1.size)
for (i in 0 until p0.size) {
    log_returns[i] = log10(p1[i]) - log10(p[i])
}
println(Arrays.toString(log_returns))

// run the Ljung-Box test
val lb_test = LjungBox(log_returns, 14, 0)
println(String.format("The null hypothesis is: %s%n", lb_test.getNullHypothesis()))
println(String.format("The alternative hypothesis is: %s%n", lb_test.getAlternativeHypothesis()))
println(String.format("The test statistic = %f%n", lb_test.statistics()))
println(String.format("The p-value = %f%n", lb_test.pValue()))

val adf = AugmentedDickeyFuller(spx.toArray())
println(adf.getNullHypothesis())
println(adf.getAlternativeHypothesis())
println(String.format("the p-value for the test = %f%n", adf.pValue()))
println(String.format("the statistics for the test = %f%n", adf.statistics()))

// auto fit an ARIMA model to S&P 500 prices
val fit_sp500_1 = AutoARIMAFit(spx.toArray(), 5, 1, 5, 0, 0, 1000)
val arima1 = fit_sp500_1.optimalModelByAIC()
val arima2 = fit_sp500_1.optimalModelByAICC()
println("The optimal model for S&P500 prices by AIC is:")
println(arima1)
println("The optimal model for S&P500 prices by AICc is:")
println(arima2)

// auto fit an ARIMA model to S&P 500 log returns
val fit_sp500_2 = AutoARIMAFit(log_returns, 5, 1, 5, 0, 0, 1000)
val arima3 = fit_sp500_2.optimalModelByAIC()
val arima4 = fit_sp500_2.optimalModelByAICC()
println("The optimal model for S&P500 log returns by AIC is:")
println(arima3)
println("The optimal model for S&P500 log returns by AICc is:")
println(arima4)

// make price forecasts based on the fitted model
val forecast_prices = ARIMAForecast(spx, arima1)
println("The next 10 price forecasts for sp500 using the ARIMA(2,1,1) model is:")
for (i in 0 until 10) {
    println(forecast_prices.next())
}

// make log return forecasts based on the fitted model
val forecast_returns = ARIMAForecast(SimpleTimeSeries(log_returns), arima4)
println("The next 10 log return forecasts for sp500 using the ARMA(2, 3) model is:")
for (i in 0 until 10) {
    println(forecast_returns.next())
}

// compute the conditional means of the model at each time
val log_returns_hat = ARMAForecastOneStep(log_returns, arima4.getARMA())

// compute the residuals = observations - fitted values
var residuals: DoubleArray = DoubleArray(log_returns.size)
for (t in 0 until log_returns.size) {
    residuals[t] = log_returns[t] - log_returns_hat.xHat(t)
}
println("residuals:")
println(Arrays.toString(residuals))

// fit the residuals to a GARCH(1,1) model
val garch_fit = GARCHFit(residuals, 1, 1)
val garch = garch_fit.getModel()
println(String.format("the residual GARCH(1,1) model is: %s%n", garch))

// fit both the ARMA and GARCH models in one go
val arma_garch_fit = ARMAGARCHFit(
                log_returns, // the input series
                2, // the order for AR in ARMA model
                3, // the order for MA in ARMA model
                1, // the order for GARCH in GARCH model
                1 // the order for ARCH in GARCH model
        )
println("the ARMA model is:")
println(arma_garch_fit.getARMAGARCHModel().getARMAModel())
println("the GARCH model is:")
println(arma_garch_fit.getARMAGARCHModel().getGARCHModel())

In [ ]:
println("Univariate time series")

// construct a time series
var ts1 = SimpleTimeSeries(doubleArrayOf(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0))
println("ts1:" + ts1)

// construct a time series by taking the first difference of another one
// each term is the difference between two successive terms in the original time series
val ts2 = ts1.diff(1)
println("ts2:" + ts2)

// construct a time series by dropping the first two terms of another one
val ts3 = ts1.drop(2)
println("ts3:" + ts3)

// construct a time series by lagging another one
// This operation makes sense only for equi-distant data points.
val ts4 = ts1.lag(2)
println("ts4:" + ts4)